In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


/kaggle/input/playground-series-s3e23/sample_submission.csv
/kaggle/input/playground-series-s3e23/train.csv
/kaggle/input/playground-series-s3e23/test.csv


In [2]:
#X = pd.read_csv('data/train.csv', index_col='id')
#X_test = pd.read_csv('data/test.csv', index_col='id')
X = pd.read_csv('/kaggle/input/playground-series-s3e23/train.csv', index_col='id')
X_test = pd.read_csv('/kaggle/input/playground-series-s3e23/test.csv', index_col='id')
y = X.pop('defects')

In [3]:
X_test.head()

,loc,v(g),ev(g),iv(g),n,v,l,d,i,e,...,t,lOCode,lOComment,lOBlank,locCodeAndComment,uniq_Op,uniq_Opnd,total_Op,total_Opnd,branchCount
id,,,,,,,,,,,,,,,,,,,,,
101763,33.0,5.0,1.0,4.0,144.0,824.82,0.04,26.96,30.05,22636.74,...,1257.60,30,0,3,0,21.0,23.0,87.0,57.0,9.0
101764,27.0,8.0,8.0,2.0,125.0,646.24,0.04,22.82,27.22,14482.46,...,804.58,23,0,2,0,18.0,19.0,70.0,49.0,15.0
101765,130.0,11.0,7.0,10.0,545.0,3831.40,0.02,48.15,66.17,116160.08,...,6453.34,99,9,17,1,26.0,53.0,333.0,244.0,21.0
101766,65.0,7.0,1.0,7.0,156.0,855.71,0.06,17.23,49.89,16135.47,...,896.42,45,8,10,0,15.0,26.0,88.0,60.0,13.0
101767,22.0,3.0,1.0,3.0,52.0,238.42,0.10,9.60,26.70,2624.49,...,145.80,16,0,4,0,12.0,15.0,30.0,24.0,5.0


## Feature Selection

TODO 
- feature comparison train/test
- nan removal/inference 

## Model 

I will first try some XGBoost model and see how it performs.

Work coming after will be based on features selection/engineering

In [4]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

### Model training

- We use "tree_method='gpu_hist'" in order to use the GPU for training.

In [11]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
import optuna
def objective_xg(trial):
    """Define the objective function"""

    params = {
        'booster': trial.suggest_categorical('booster', ['gbtree']),
        'max_depth': trial.suggest_int('max_depth', 1, 10),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.05),
        'n_estimators': trial.suggest_int('n_estimators', 300, 700),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'gamma': trial.suggest_loguniform('gamma', 1e-8, 1.0),
        'subsample': trial.suggest_loguniform('subsample', 0.1, 1.0),
        'colsample_bytree': trial.suggest_loguniform('colsample_bytree', 0.01, 0.5),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-8, 1.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-8, 1.0),
        'eval_metric': 'mlogloss',
        'use_label_encoder': False
    }
    model_xgb = XGBClassifier(**params, tree_method='gpu_hist')
    model_xgb.fit(X_train, y_train)
    y_pred = model_xgb.predict(X_val)
    return accuracy_score(y_val,y_pred)

In [12]:
study_xgb = optuna.create_study(direction='maximize')
optuna.logging.set_verbosity(optuna.logging.WARNING)
study_xgb.optimize(objective_xg, n_trials=50,show_progress_bar=True)

  0%|          | 0/50 [00:00<?, ?it/s]

In [16]:
xgb = XGBClassifier(**study_xgb.best_params, tree_method='gpu_hist')
xgb.fit(X_train, y_train)
y_pred = xgb.predict(X_val)
print('Accuracy: ', accuracy_score(y_val, y_pred))

Accuracy:  0.8151623839237458


In [18]:
xgb = XGBClassifier(**study_xgb.best_params, tree_method='gpu_hist')
xgb.fit(X_train, y_train)
y_prob = xgb.predict_proba(X_val)[:, 1]
y_prob

array([0.15500951, 0.28823283, 0.34397945, ..., 0.6182461 , 0.12972496,
       0.70365494], dtype=float32)

In [21]:
#y_pred = xgb.predict(X_test)
y_prob = xgb.predict_proba(X_test)[:, 1]
submission = pd.DataFrame({'id': X_test.index, 'defects': y_prob})
#submission.to_csv('data/submission.csv', index=False)
submission.to_csv('submission.csv', index=False)
submission.head()

,id,defects
0,101763,0.245804
1,101764,0.201940
2,101765,0.635411
3,101766,0.487310
4,101767,0.140139
